In [1]:
import pandas as pd
import numpy as np
import re

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import StratifiedKFold

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.optimizers import Adam

from keras.utils import np_utils

import warnings 
warnings.filterwarnings(action='ignore')

In [3]:
train = pd.read_csv("data/train_data.csv",encoding="utf-8",index_col=False)
test = pd.read_csv("data/test_data.csv",index_col=False)
submission = pd.read_csv("data/sample_submission.csv")

In [5]:
from konlpy.tag import Mecab  
tokenizer = Mecab()
train["tokenized"] = [tokenizer.morphs(sentence) for sentence in train["title"]]
test["tokenized"] = [tokenizer.morphs(sentence) for sentence in test["title"]]

In [6]:
train.head()

,index,title,topic_idx,tokenized
0,0,인천→핀란드 항공기 결항…휴가철 여행객 분통,4,"[인천, →, 핀란드, 항공기, 결항, …, 휴가철, 여행객, 분통]"
1,1,실리콘밸리 넘어서겠다…구글 15조원 들여 美전역 거점화,4,"[실리콘밸리, 넘어서, 겠, 다, …, 구글, 15, 조, 원, 들여, 美, 전역,..."
2,2,이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것,4,"[이란, 외무, 긴장, 완화, 해결책, 은, 미국, 이, 경제, 전쟁, 멈추, 는, 것]"
3,3,NYT 클린턴 측근韓기업 특수관계 조명…공과 사 맞물려종합,4,"[NYT, 클린턴, 측근, 韓, 기업, 특수, 관계, 조명, …, 공과, 사, 맞물..."
4,4,시진핑 트럼프에 중미 무역협상 조속 타결 희망,4,"[시진핑, 트럼프, 에, 중미, 무역, 협상, 조속, 타결, 희망]"


In [7]:
test.head()

,index,title,tokenized
0,45654,유튜브 내달 2일까지 크리에이터 지원 공간 운영,"[유튜브, 내달, 2, 일, 까지, 크리에이터, 지원, 공간, 운영]"
1,45655,어버이날 맑다가 흐려져…남부지방 옅은 황사,"[어버이날, 맑, 다가, 흐려져, …, 남부, 지방, 옅, 은, 황사]"
2,45656,내년부터 국가RD 평가 때 논문건수는 반영 않는다,"[내년, 부터, 국가, RD, 평가, 때, 논문, 건수, 는, 반영, 않, 는다]"
3,45657,김명자 신임 과총 회장 원로와 젊은 과학자 지혜 모을 것,"[김명자, 신임, 과, 총, 회장, 원로, 와, 젊, 은, 과학자, 지혜, 모을, 것]"
4,45658,회색인간 작가 김동식 양심고백 등 새 소설집 2권 출간,"[회색, 인간, 작가, 김동식, 양, 심, 고, 백, 등, 새, 소설, 집, 2, ..."


In [8]:
for tokenized in train["tokenized"]:
    for token in tokenized:
        if len(token) == 1:
            tokenized.remove(token)  
            
for tokenized in test["tokenized"]:
    for token in tokenized:
        if len(token) == 1:
            tokenized.remove(token)   


In [9]:
train.head()

,index,title,topic_idx,tokenized
0,0,인천→핀란드 항공기 결항…휴가철 여행객 분통,4,"[인천, 핀란드, 항공기, 결항, 휴가철, 여행객, 분통]"
1,1,실리콘밸리 넘어서겠다…구글 15조원 들여 美전역 거점화,4,"[실리콘밸리, 넘어서, 다, 구글, 15, 원, 들여, 전역, 거점]"
2,2,이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것,4,"[이란, 외무, 긴장, 완화, 해결책, 미국, 경제, 전쟁, 멈추, 것]"
3,3,NYT 클린턴 측근韓기업 특수관계 조명…공과 사 맞물려종합,4,"[NYT, 클린턴, 측근, 기업, 특수, 관계, 조명, 공과, 맞물려, 종합]"
4,4,시진핑 트럼프에 중미 무역협상 조속 타결 희망,4,"[시진핑, 트럼프, 중미, 무역, 협상, 조속, 타결, 희망]"


In [10]:
test.head()

,index,title,tokenized
0,45654,유튜브 내달 2일까지 크리에이터 지원 공간 운영,"[유튜브, 내달, 일, 까지, 크리에이터, 지원, 공간, 운영]"
1,45655,어버이날 맑다가 흐려져…남부지방 옅은 황사,"[어버이날, 다가, 흐려져, 남부, 지방, 은, 황사]"
2,45656,내년부터 국가RD 평가 때 논문건수는 반영 않는다,"[내년, 부터, 국가, RD, 평가, 논문, 건수, 반영, 는다]"
3,45657,김명자 신임 과총 회장 원로와 젊은 과학자 지혜 모을 것,"[김명자, 신임, 총, 회장, 원로, 젊, 과학자, 지혜, 모을]"
4,45658,회색인간 작가 김동식 양심고백 등 새 소설집 2권 출간,"[회색, 인간, 작가, 김동식, 심, 백, 새, 소설, 2, 출간]"


In [22]:
vocab_list = pd.concat([train,test])
vocab_list = vocab_list["tokenized"]
vocab_list[:20]

0                      [인천, 핀란드, 항공기, 결항, 휴가철, 여행객, 분통]
1                [실리콘밸리, 넘어서, 다, 구글, 15, 원, 들여, 전역, 거점]
2              [이란, 외무, 긴장, 완화, 해결책, 미국, 경제, 전쟁, 멈추, 것]
3           [NYT, 클린턴, 측근, 기업, 특수, 관계, 조명, 공과, 맞물려, 종합]
4                    [시진핑, 트럼프, 중미, 무역, 협상, 조속, 타결, 희망]
5                  [팔레스타인, 자, 구서, 16, 소년, 이스라엘, 총격, 사망]
6              [인도, 48, 만, 파키스탄, 공습, 테러, 캠프, 폭격, 종합, 보]
7     [대선, TV, 토론, 음담패설, 만회, 실패, 트럼프, 사과, 대신, 빌클린턴, ...
8               [푸틴, 한반도, 상황, 진전, 위한, 방안, 김정은, 위원장, 논의]
9             [특검, 면죄부, 은, 트럼프, 스캔들, 보도, 언론, 맹공, 국민, 적]
10                      [오키, 나와서, 열린, 강제, 징용, 노동자, 추도식]
11               [이란, 최고, 지도자, 모욕, 혐의, 미국인, 징역, 10, 선고]
12                [카니발, 축제, 러, 자, 브라질, 리우, 대형, 유람선, 행렬]
13               [올랜도, 병원, 최악, 총기, 테러, 부상자, 치료비, 받, 는다]
14                       [대, 기업, 올해, 평균, ., 46, 임금, 인상]
15                    [WMO, 엘니뇨, 여전히, 강력, 2, 분기, 소멸, 듯]
16                [이스라엘, 네타냐후, 유대교, 병역, 문제, 연정, 협상, 진통]
17                  [UAE, 사우디, 어, 호르무즈, 호위, 연합, 

In [23]:
from nltk import FreqDist
vocab = FreqDist(np.hstack(vocab_list))
print('단어 집합의 크기 : {}'.format(len(vocab)))

단어 집합의 크기 : 31593


In [24]:
vocab_size = 5000
# 상위 vocab_size개의 단어만 보존
vocab = vocab.most_common(vocab_size)
print('단어 집합의 크기 : {}'.format(len(vocab)))

단어 집합의 크기 : 5000


In [25]:
word_to_index = {word[0] : index + 2 for index, word in enumerate(vocab)}
word_to_index['pad'] = 1
word_to_index['unk'] = 0

In [26]:
train_x = []
test_x = []

for line in train["tokenized"]: #입력 데이터에서 1줄씩 문장을 읽음
    temp = []
    for w in line: #각 줄에서 1개씩 글자를 읽음
        try:
            temp.append(word_to_index[w]) # 글자를 해당되는 정수로 변환
        except KeyError: # 단어 집합에 없는 단어일 경우 unk로 대체된다.
            temp.append(word_to_index['unk']) # unk의 인덱스로 변환

    train_x.append(temp)

for line in test["tokenized"]: #입력 데이터에서 1줄씩 문장을 읽음
    temp = []
    for w in line: #각 줄에서 1개씩 글자를 읽음
        try:
            temp.append(word_to_index[w]) # 글자를 해당되는 정수로 변환
        except KeyError: # 단어 집합에 없는 단어일 경우 unk로 대체된다.
            temp.append(word_to_index['unk']) # unk의 인덱스로 변환

    test_x.append(temp)

In [27]:
print(train_x[:20])
print(test_x[:20])

[[334, 0, 2535, 4771, 0, 2962, 0], [0, 0, 18, 295, 154, 9, 0, 1615, 2963], [54, 697, 985, 746, 0, 77, 84, 402, 4567, 125], [3582, 2657, 2964, 62, 1498, 450, 2247, 0, 0, 2], [550, 32, 0, 397, 202, 0, 2478, 793], [1649, 280, 0, 266, 2888, 254, 1207, 99], [477, 2061, 26, 2062, 1185, 241, 1262, 2889, 2, 15], [296, 335, 1475, 0, 0, 677, 32, 936, 538, 0, 174, 0], [812, 589, 766, 2353, 324, 960, 138, 145, 169], [710, 0, 13, 32, 3123, 339, 159, 3436, 85, 857], [0, 0, 1852, 1853, 3437, 551, 0], [54, 72, 961, 0, 455, 2536, 1762, 14, 1944], [0, 106, 678, 280, 803, 4171, 1291, 4772, 3215], [3836, 986, 1443, 1730, 241, 0, 0, 1499, 282], [95, 62, 68, 568, 16, 1894, 1071, 380], [0, 0, 1794, 643, 25, 47, 0, 215], [254, 1135, 0, 3708, 210, 1561, 202, 3124], [1263, 285, 30, 1650, 4172, 776, 393, 906], [285, 3583, 1035, 174, 987, 711, 804, 51, 1019], [0, 21, 0, 0, 0, 17, 320, 2658, 976]]
[[1399, 199, 57, 33, 0, 53, 1390, 333], [0, 1402, 0, 841, 414, 13, 3092], [133, 45, 166, 1097, 505, 2640, 4333, 2253, 

In [28]:
max_len = max(len(l) for l in train_x)
max_len = max(len(l) for l in test_x)
print(max_len)

18


In [30]:
for line in train_x:
    if len(line) < max_len: # 현재 샘플이 정해준 길이보다 짧으면
        line += [word_to_index['pad']] * (max_len - len(line)) # 나머지는 전부 'pad' 토큰으로 채운다.
        
for line in test_x:
    if len(line) < max_len: # 현재 샘플이 정해준 길이보다 짧으면
        line += [word_to_index['pad']] * (max_len - len(line)) # 나머지는 전부 'pad' 토큰으로 채운다.

In [34]:
print('리뷰의 최대 길이 : %d' % max(len(l) for l in train_x))
print('리뷰의 최소 길이 : %d' % min(len(l) for l in train_x))
print('리뷰의 평균 길이 : %f' % (sum(map(len, train_x))/len(train_x)))
      
print('리뷰의 최대 길이 : %d' % max(len(l) for l in test_x))
print('리뷰의 최소 길이 : %d' % min(len(l) for l in test_x))
print('리뷰의 평균 길이 : %f' % (sum(map(len, test_x))/len(test_x)))

리뷰의 최대 길이 : 18
리뷰의 최소 길이 : 18
리뷰의 평균 길이 : 18.000000
리뷰의 최대 길이 : 18
리뷰의 최소 길이 : 18
리뷰의 평균 길이 : 18.000000


In [16]:
# 종속변수 데이터 전처리
train_y = np_utils.to_categorical(train["topic_idx"]) # Y_train 에 원-핫 인코딩
print(train_y)
print(train_y.shape)

[[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]]
(45654, 7)


In [17]:
len(encoded)

45654

In [35]:
train_x = np.array(train_x)
test_x = np.array(test_x)
train_x

array([[ 334,    0, 2535, ...,    1,    1,    1],
       [   0,    0,   18, ...,    1,    1,    1],
       [  54,  697,  985, ...,    1,    1,    1],
       ...,
       [  23, 1880,  968, ...,    1,    1,    1],
       [2989,    4,    0, ...,    1,    1,    1],
       [ 968,    8,  244, ...,    1,    1,    1]])

In [36]:
#파라미터 설정
vocab_size = 5003 # 제일 많이 사용하는 사이즈
embedding_dim = 200  
max_length = 18    # 위에서 그래프 확인 후 정함
padding_type='post'
#oov_tok = "<OOV>"

In [37]:
# 양방향 LSTM 레이어를 사용한 모델 (model3) 정의
model = Sequential([Embedding(vocab_size, embedding_dim, input_length =max_length),
        tf.keras.layers.Bidirectional(LSTM(units = 64, return_sequences = True)),
        tf.keras.layers.Bidirectional(LSTM(units = 64, return_sequences = True)),
        tf.keras.layers.Bidirectional(LSTM(units = 64)),
        Dense(7, activation='softmax')    # 결과값이 0~4 이므로 Dense(5)
    ])
    
model.compile(loss= 'categorical_crossentropy', #여러개 정답 중 하나 맞추는 문제이므로 손실 함수는 categorical_crossentropy
              optimizer= 'adam',
              metrics = ['accuracy']) 
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 18, 200)           1000600   
_________________________________________________________________
bidirectional (Bidirectional (None, 18, 128)           135680    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 18, 128)           98816     
_________________________________________________________________
bidirectional_2 (Bidirection (None, 128)               98816     
_________________________________________________________________
dense (Dense)                (None, 7)                 903       
Total params: 1,334,815
Trainable params: 1,334,815
Non-trainable params: 0
_________________________________________________________________


In [21]:
# # 모델 실행해보기
# history = model.fit(train_x, train_y, epochs=10, batch_size=100, validation_split= 0.2) 
#   # 양방향 LSTM 레이어에서는 batch size 를 100으로 잡고 50회 학습 해보았다.

In [22]:
# # 모델 학습 결과 확인
# plt.figure(figsize=(12, 4))

# plt.subplot(1, 2, 1)
# plt.title('loss of Bidirectional LSTM (model3) ', fontsize= 15)
# plt.plot(history.history['loss'], 'b-', label='loss')
# plt.plot(history.history['val_loss'],'r--', label='val_loss')
# plt.xlabel('Epoch')
# plt.legend()

# plt.subplot(1, 2, 2)
# plt.title('accuracy of Bidirectional LSTM (model3) ', fontsize= 15)
# plt.plot(history.history['accuracy'], 'g-', label='accuracy')
# plt.plot(history.history['val_accuracy'],'k--', label='val_accuracy')
# plt.xlabel('Epoch')
# plt.legend()
# plt.show

In [39]:
train_y = train["topic_idx"]
len(train_x),len(train_y)

(45654, 45654)

In [24]:
# # 계층 교차 검증
# n_fold = 5  
# seed = 42

# cv = StratifiedKFold(n_splits = n_fold, shuffle=True, random_state=seed)

# for i, (i_trn, i_val) in enumerate(cv.split(train_x, train["topic_idx"]), 1):
#     print(f'training model for CV #{i}')

#     model.fit(train_x[i_trn], 
#             to_categorical(train["topic_idx"][i_trn]),
#             validation_data=(train_x[i_val], to_categorical(train["topic_idx"][i_val])),
#             epochs=10,
#             batch_size=512)

training model for CV #1
Epoch 1/10
72/72 [==============================] - 16s 158ms/step - loss: 1.2046 - accuracy: 0.5346 - val_loss: 0.6425 - val_accuracy: 0.7873
Epoch 2/10
72/72 [==============================] - 9s 131ms/step - loss: 0.5211 - accuracy: 0.8286 - val_loss: 0.5559 - val_accuracy: 0.8138
Epoch 3/10
72/72 [==============================] - 10s 134ms/step - loss: 0.4111 - accuracy: 0.8687 - val_loss: 0.5625 - val_accuracy: 0.8160
Epoch 4/10
72/72 [==============================] - 10s 133ms/step - loss: 0.3538 - accuracy: 0.8869 - val_loss: 0.5839 - val_accuracy: 0.8097
Epoch 5/10
72/72 [==============================] - 10s 133ms/step - loss: 0.3159 - accuracy: 0.8988 - val_loss: 0.6201 - val_accuracy: 0.8058
Epoch 6/10
72/72 [==============================] - 10s 134ms/step - loss: 0.2805 - accuracy: 0.9115 - val_loss: 0.6290 - val_accuracy: 0.8102
Epoch 7/10
72/72 [==============================] - 10s 134ms/step - loss: 0.2534 - accuracy: 0.9207 - val_loss: 0.666

In [41]:
# 계층 교차 검증
n_fold = 5  
seed = 42

cv = StratifiedKFold(n_splits = n_fold, shuffle=True, random_state=seed)

# 테스트데이터의 예측값 담을 곳 생성
test_y = np.zeros((test_x.shape[0], 7))

# 조기 종료 옵션 추가
es = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3,
                   verbose=1, mode='min', baseline=None, restore_best_weights=True)

for i, (i_trn, i_val) in enumerate(cv.split(train_x, train_y), 1):
    print(f'training model for CV #{i}')

    model.fit(train_x[i_trn], 
            to_categorical(train_y[i_trn]),
            validation_data=(train_x[i_val], to_categorical(train_y[i_val])),
            epochs=10,
            batch_size=512,
            callbacks=[es])     # 조기 종료 옵션
                      
    test_y += model.predict(test_x) / n_fold    # 나온 예측값들을 교차 검증 횟수로 나눈다

training model for CV #1
Epoch 1/10
72/72 [==============================] - 16s 163ms/step - loss: 1.2244 - accuracy: 0.5197 - val_loss: 0.6411 - val_accuracy: 0.7820
Epoch 2/10
72/72 [==============================] - 10s 144ms/step - loss: 0.5107 - accuracy: 0.8330 - val_loss: 0.5505 - val_accuracy: 0.8161
Epoch 3/10
72/72 [==============================] - 10s 144ms/step - loss: 0.3915 - accuracy: 0.8748 - val_loss: 0.5647 - val_accuracy: 0.8206
Epoch 4/10
72/72 [==============================] - 10s 145ms/step - loss: 0.3413 - accuracy: 0.8889 - val_loss: 0.5892 - val_accuracy: 0.8131
Epoch 5/10
72/72 [==============================] - 11s 149ms/step - loss: 0.2979 - accuracy: 0.9036 - val_loss: 0.6150 - val_accuracy: 0.8092
Restoring model weights from the end of the best epoch.
Epoch 00005: early stopping
training model for CV #2
Epoch 1/10
72/72 [==============================] - 11s 148ms/step - loss: 0.4340 - accuracy: 0.8601 - val_loss: 0.4307 - val_accuracy: 0.8592
Epoch 2/

In [42]:
test_y

array([[0.08602009, 0.0717154 , 0.45507176, ..., 0.04346492, 0.01626402,
        0.0102361 ],
       [0.00300351, 0.0018623 , 0.02537468, ..., 0.00472795, 0.00252248,
        0.00157376],
       [0.05173881, 0.03597972, 0.7157121 , ..., 0.01904189, 0.00242945,
        0.16792241],
       ...,
       [0.00204276, 0.00104967, 0.04095534, ..., 0.02297982, 0.00180785,
        0.00299678],
       [0.31421448, 0.04062183, 0.55364918, ..., 0.03585994, 0.00165979,
        0.0270768 ],
       [0.0069064 , 0.01110192, 0.29724516, ..., 0.00847807, 0.00150495,
        0.67217722]])

In [43]:
topic = []
for i in range(len(test_y)):
    topic.append(np.argmax(test_y[i]))

In [44]:
submission['topic_idx'] = topic
submission

,index,topic_idx
0,45654,2
1,45655,3
2,45656,2
3,45657,2
4,45658,3
...,...,...
9126,54780,3
9127,54781,6
9128,54782,3
9129,54783,2


In [45]:
submission.to_csv('results/kfold-LSTM-2.csv',index = False)

In [46]:
test_y

array([[0.08602009, 0.0717154 , 0.45507176, ..., 0.04346492, 0.01626402,
        0.0102361 ],
       [0.00300351, 0.0018623 , 0.02537468, ..., 0.00472795, 0.00252248,
        0.00157376],
       [0.05173881, 0.03597972, 0.7157121 , ..., 0.01904189, 0.00242945,
        0.16792241],
       ...,
       [0.00204276, 0.00104967, 0.04095534, ..., 0.02297982, 0.00180785,
        0.00299678],
       [0.31421448, 0.04062183, 0.55364918, ..., 0.03585994, 0.00165979,
        0.0270768 ],
       [0.0069064 , 0.01110192, 0.29724516, ..., 0.00847807, 0.00150495,
        0.67217722]])

In [47]:
test_y_df = pd.DataFrame(test_y)

In [49]:
test_y_df.to_csv('ensemble/kfold-lstm.csv')

NameError: name 'test_y_df' is not defined